In [2]:
import pandas as pd
import os
import requests
from tqdm import tqdm

In [ ]:
file_path = "Books-BX-10.csv"
df = pd.read_csv(file_path, delimiter=";", encoding="latin1")

In [6]:
print(df.head())

         ISBN                                              Title  \
0  0002005018                                       Clara Callan   
1  0374157065  Flu: The Story of the Great Influenza Pandemic...   
2  0399135782                             The Kitchen God's Wife   
3  0440234743                                      The Testament   
4  0452264464               Beloved (Plume Contemporary Fiction)   

                 Author  Year              Publisher  \
0  Richard Bruce Wright  2001  HarperFlamingo Canada   
1      Gina Bari Kolata  1999   Farrar Straus Giroux   
2               Amy Tan  1991       Putnam Pub Group   
3          John Grisham  1999                   Dell   
4         Toni Morrison  1994                  Plume   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0002005018.0...   
1  http://images.amazon.com/images/P/0374157065.0...   
2  http://images.amazon.com/images/P/0399135782.0...   
3  http://images.amazon.com/im

In [7]:
def get_best_image_url(row):
    """Returns the best available image URL (Large > Medium > Small)"""
    return row['Image-URL-L'] if pd.notna(row['Image-URL-L']) else (
        row['Image-URL-M'] if pd.notna(row['Image-URL-M']) else row['Image-URL-S']
    )

df['Best-Image-URL'] = df.apply(get_best_image_url, axis=1)

In [8]:
print(f"Number of rows: {df.shape[0]}")

Number of rows: 13581


In [9]:
!pip install tensorflow tensorflow_hub

In [10]:
from tqdm import tqdm
import requests
from PIL import Image
from io import BytesIO
import tensorflow as tf
import numpy as np
import pandas as pd

inception_v3_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', pooling='avg')

def preprocess_image(image_url):
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        response = requests.get(image_url, timeout=20, headers=headers, allow_redirects=True)

        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            img = img.resize((299, 299))
            img_array = np.array(img) / 255.0

            if img_array.shape != (299, 299, 3):
                img_array = np.stack([img_array] * 3, axis=-1)

            img_array = np.expand_dims(img_array, axis=0)
            img_array = tf.keras.applications.inception_v3.preprocess_input(img_array)
            return img_array
    except Exception:
        return None

def extract_visual_features(image_url):
    img_array = preprocess_image(image_url)
    if img_array is not None:
        features = inception_v3_model.predict(img_array)
        return features.flatten()
    return None

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
file_path = "Books_BX_15_10.csv"
df_existing_features = pd.read_csv(file_path, delimiter=",", encoding="latin1")
df_existing_features.set_index("ISBN", inplace=True)
print(df_existing_features.head())

            Unnamed: 0                                              Title  \
ISBN                                                                        
0002005018           1                                       Clara Callan   
0374157065           3  Flu: The Story of the Great Influenza Pandemic...   
0399135782           5                             The Kitchen God's Wife   
0440234743          18                                      The Testament   
0452264464          19               Beloved (Plume Contemporary Fiction)   

                          Author  Year              Publisher  \
ISBN                                                            
0002005018  Richard Bruce Wright  2001  HarperFlamingo Canada   
0374157065      Gina Bari Kolata  1999   Farrar Straus Giroux   
0399135782               Amy Tan  1991       Putnam Pub Group   
0440234743          John Grisham  1999                   Dell   
0452264464         Toni Morrison  1994                  Plume   

    

In [ ]:
total_books = df.shape[0]
progress_interval = total_books // 100

visual_features_list = []
books_without_url_count = 0

with tqdm(total=total_books, desc="Extracting Features", unit="book") as pbar:
    for idx, row in df.iterrows():
        best_image_url = get_best_image_url(row)

        if pd.notna(best_image_url):
            features = extract_visual_features(best_image_url)
            if features is not None:
                visual_features_list.append(features)
            else:
                books_without_url_count += 1
        else:
            books_without_url_count += 1

        if idx % progress_interval == 0:
            pbar.update(progress_interval)

visual_features_df = pd.DataFrame(visual_features_list)

df_with_features = df.iloc[:len(visual_features_list)].copy()
df_with_features = pd.concat([df_with_features, visual_features_df], axis=1)

output_path = "Books_with_Visual_Features.csv"
df_with_features.to_csv(output_path, index=False)

print(f"Number of books without a functioning image URL: {books_without_url_count}")
print(f"Visual features saved at: {output_path}")

Extracting Features:   0%|          | 0/17480 [00:00<?, ?book/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step


Extracting Features:   1%|          | 174/17480 [00:00<00:39, 436.49book/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 591ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

Extracting Features:   1%|          | 174/17480 [00:17<00:39, 436.49book/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

Extracting Features: 17574book [4:50:39,  1.03s/book]                         

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

Extracting Features: 17574book [4:51:59,  1.00book/s]


Number of books without a functioning image URL: 4
Visual features saved at: /content/drive/MyDrive/IR Assignment/Books_with_Visual_Features.csv


In [ ]:
# With lookup of existing visual features
total_books = df.shape[0]
progress_interval = max(1, total_books // 100)

visual_features_list = []
books_without_url_count = 0

with tqdm(total=total_books, desc="Processing Books", unit="book") as pbar:
    for idx, row in df.iterrows():
        isbn = row["ISBN"]

        if isbn in df_existing_features.index:
            features = df_existing_features.loc[isbn, '0':'2047'].values
        else:
            best_image_url = get_best_image_url(row)
            if pd.notna(best_image_url):
                features = extract_visual_features(best_image_url)
                if features is None:
                    books_without_url_count += 1
            else:
                features = None
                books_without_url_count += 1

        if features is not None:
            visual_features_list.append(features)
        else:
            visual_features_list.append([None] * 2048)

        if idx % progress_interval == 0:
            pbar.update(progress_interval)

visual_features_df = pd.DataFrame(visual_features_list, columns=[str(i) for i in range(2048)])

df_with_features = pd.concat([df, visual_features_df], axis=1)

output_path = "Books_with_Visual_Features.csv"

df_with_features.to_csv(output_path, index=False)

print(f"Number of books without a functioning image URL: {books_without_url_count}")
print(f"Visual features saved at: {output_path}")

Processing Books:   0%|          | 0/13581 [00:00<?, ?book/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step


Processing Books:   2%|▏         | 270/13581 [00:03<03:12, 69.27book/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step


Processing Books:   3%|▎         | 405/13581 [00:05<02:49, 77.84book/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 510ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

Processing Books: 13635book [13:31,  3.94book/s]                         

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

Processing Books: 13635book [14:04, 16.15book/s]


Number of books without a functioning image URL: 0
Visual features saved at: /content/drive/MyDrive/IR Assignment/Books_with_Visual_Features.csv
